In [1]:
from datetime import datetime as dt
import os
import sqlite3
import shutil
from tqdm.notebook import tqdm
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive

gauth = GoogleAuth()
gauth.CommandLineAuth()
drive = GoogleDrive(gauth)

In [8]:
conn = sqlite3.connect('../data/model_results.db')
c = conn.cursor()
c.execute('''
SELECT * FROM results 
WHERE (true_label IS NOT NULL
AND NOT (true_label = "none"))
OR (NOT (true_label = prediction)
OR confidence < 0.75);
''')
rows = c.fetchall()

In [9]:
# c.execute('''CREATE TABLE training (
#     utc_datetime text NOT NULL PRIMARY KEY,
#     datetime text NOT NULL, 
#     file_name text NOT NULL, 
#     predicton text NOT NULL, 
#     confidence real NOT NULL, 
#     true_label text)''')
# conn.commit()

In [10]:
# Number of new rows for training data
len(rows)

17653

In [22]:
training_folder_id = '1ABP5b7r6ac6bVENGWEobN6wdvRw5Hj1c'
# Auto-iterate through all files that matches this query
gdrive_file_list = drive.ListFile({'q': f"'{training_folder_id}' in parents"}).GetList()
gdrive_file_list = [f['title'] for f in gdrive_file_list]

In [23]:
new_training_imgs = [r for r in rows if r[5]+'_'+r[1]+'.jpg' not in gdrive_file_list]
len(new_training_imgs)

0

In [21]:
# Iterate over the results above and upload to google drive training folder
for i, row in tqdm(enumerate(new_training_imgs), total=len(new_training_imgs)):
    utc_datetime, datetime, file_name, prediction, confidence, label, _ = row
    gdrive_fname = f'{label}_{datetime}.jpg'
    f = drive.CreateFile({'title': gdrive_fname, 'parents': [{'id': training_folder_id}]})
    f.SetContentFile(f'../imgs/{file_name}')
    try:
        f.Upload()
    except:
        # Try a second time if the connection errors out
        f.Upload()

  0%|          | 0/1038 [00:00<?, ?it/s]

In [24]:
conn.commit()
conn.close()